# Chess Model
This notebook focuses on creating a predictive model for the winning side of a chess match given information about the number of moves.

## Import Statements and Data
In this section we'll import our packages we'll be using and import our data. We'll also check for data quality.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from IPython.display import clear_output
import os

Reading in our dataset and setting it to a variable called games.

In [2]:
games = pd.read_csv("chess_games.csv")

Let's check the data quality. We want to make sure we aren't workin with any null data.

In [3]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6256184 entries, 0 to 6256183
Data columns (total 15 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Event            object 
 1   White            object 
 2   Black            object 
 3   Result           object 
 4   UTCDate          object 
 5   UTCTime          object 
 6   WhiteElo         int64  
 7   BlackElo         int64  
 8   WhiteRatingDiff  float64
 9   BlackRatingDiff  float64
 10  ECO              object 
 11  Opening          object 
 12  TimeControl      object 
 13  Termination      object 
 14  AN               object 
dtypes: float64(2), int64(2), object(11)
memory usage: 716.0+ MB


In [4]:
#games['rating_diff'] = games['white_rating'] - games['black_rating']

In [5]:
games = games.sample(frac=1).reset_index()
games = games[['Result','AN']]
#games = games.drop(columns=['rating_diff'])
#Comment to subset out draws
#games = games[games['Result'] != 'draw']

In [6]:
games = games[~(games['Result'].str.contains('*',regex=False))]

In [7]:
games['Result'] = games['Result'].apply(lambda x: int(x[0]))

In [8]:
games['winner'] = games['Result']
games = games.drop(columns=['Result'])

In [9]:
games['moves'] = games['AN']
game = games.drop(columns=['AN'])

Now that we know that our data doesn't contain nulls, let's check roughly what our baseline percent is.

In [10]:
games['winner'].value_counts(normalize=True)

1    0.535976
0    0.464024
Name: winner, dtype: float64

Looking at this, our model will need to perform above roughly 50% to be accurate.

## Data Cleaning and Preprocessing
Unfortunately, the moves columns is our only source of information for what is going on in our games turn to turn. Because of this we'll need to work vigorously to get the data the way we want it in. First things first let's split this column into the desired number of turns. This needs to be standardized so that our number of inputs for our model is always the same. We'll create a variable, turns, representing the number of turns we want.

In [11]:
turns = 30

Now we'll need to create a function to return the our moves column split and spliced to our desired size. An important issue of note is because we have to have consistent input, games with less turns will need to be padded. For ease of use, we'll pad the empty turns with 0.

In [12]:
def splitStandardize(array, length = turns):
    split_array = pd.Series(array.split(' '))[0:int(length*1.5)+1]
    split_array = list(split_array[split_array.str.contains('.',regex=False) == False])[0:length]
    while len(split_array) < length:
        split_array.append('0')
    return(split_array)

Because we have such a large dataset, in order to keep the memory managable, we'll have to batch our dataset so that it properly works. We'll batch our data into 10 sections.

In [13]:
batched = np.array_split(games,20)
test = batched[-1]
batched = batched[0:-1]
del(games)

Now we can create a new column for our first x amount of turns.

Now we need to conceptualize what we're going to do with these moves. Chess has a couple of things of note for the notation. Here's what we need to know.
* Move put opponent in check: + (++ means checkmate)
* Piece was taken in move: x
* Different notations are used for different pieces
  * K : King
  * Q : Queen
  * R : Rook
  * B : Bishop
  * N : Knight
  * P : pawn (Note that pawn is also the defualt)
* Castleing is indicated by O-O or O-O-O
* Lower case letters followed by a number represents the coordinates of the play. This get's weird as a piece was taken. If a piece was taken, the coordinates of the final location are given after the "x".

With this information we should be able to work with putting our information about moves into vectors. Each turn or move will contain:
- A feature representing the x coordinate (letters)
- A feature repressenting the y cooridnate (numbers)
- Dummy columns for each piece that was moved (other than pawn)
- A flag for if a piece was taken
- A flag for if a move resulted in a check
- A flag for if the turn was null (ie the game was finished already)
- A flag for if a castle occured

We'll need to create functions for each of these flags.

In [14]:
def flagPieceTaken(array):
    if '0' == array:
        return(-1)
    elif 'x' in array:
        return(1)
    else:
        return(0)

In [15]:
def flagCheck(array):
    if '0' == array:
        return(-1)
    elif '+' in array:
        return(1)
    else:
        return(0)

In [16]:
def flagNull(array):
    if '0' == array:
        return(1)
    else:
        return(0)

In [17]:
def flagPieceType(turnNumber,dataframe):
    dataframe['temp'] = dataframe['moves'].apply(lambda x: x[turnNumber])
    for piece in ['K','Q','R','B','N']:
        newColumnName = piece + str(turnNumber + 1)
        dataframe[newColumnName] = [0] * len(dataframe)
        dataframe.loc[dataframe['temp'].str.contains(piece,na=False),newColumnName] = 1

In [18]:
def xCoord(array, case = {'a':1,'b':2,'c':3,'d':4,'e':5,'f':6,'g':7,'h':8}):
    key = ''
    index = -1
    while (key not in case) and (abs(index) <= len(array)):
        key = array[index]
        index-= 1
    if key in case:
        return(case[key])
    else:
        return(-1)

In [19]:
def yCoord(array, case = {'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8}):
    key = ''
    index = -1
    while (key not in case) and (abs(index) <= len(array)):
        key = array[index]
        index-= 1
    if key in case:
        return(case[key])
    else:
        return(-1)

In [20]:
def castleFlag(array):
    if array == '0':
        return(-1)
    elif 'O-O' in array:
        return(1)
    else:
        return(0)

Now that we have all of our functions built, let's create one big function that combines them all. This will make our code more interpretable.

In [21]:
def combined(turnNumber,dataframe):
    newColumnName = str(turnNumber + 1)
    array = dataframe['moves'].apply(lambda x: x[turnNumber])
    array = array.apply(lambda x: x.replace('=',''))
    dataframe[f"{newColumnName}PieceTaken"] = array.apply(lambda x: flagPieceTaken(x))
    dataframe[f"{newColumnName}Check"] = array.apply(lambda x: flagCheck(x))
    dataframe[f"{newColumnName}NullTurn"] = array.apply(lambda x: flagNull(x))
    dataframe[f"{newColumnName}XCoord"] = array.apply(lambda x: xCoord(x))
    dataframe[f"{newColumnName}YCoord"] = array.apply(lambda x: yCoord(x))
    dataframe[f"{newColumnName}Castle"] = array.apply(lambda x: castleFlag(x))
    flagPieceType(turnNumber,dataframe)

Now let's loop through all the turns that we have and create these features.

In [22]:
def createTurns(games,turns=turns):
    for turnNumber in range(turns):
        combined(turnNumber,games)

Now let's check and make sure this was successful.

The fact that there are no columns names tells us that we have no columns that have na values. Let's drop our temp, moves, and columnName columns so that we can get ready to model.

In [23]:
def dropColumns(games):
    games = games.drop(columns = ['temp','moves'])

## Deep learning KEK

In [24]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

Now we create our model

In [25]:
def createModel(turns=turns):
    model = models.Sequential()
    model.add(layers.InputLayer(turns*11))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(32,activation='softmax'))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
    return(model)

In [26]:
def fitBatch(batch):
    checkpoint_path = "training_1/checkpoint.ckpt"
    model = createModel()
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
    try:
        checkpoint_dir = os.path.dirname(checkpoint_path)
        latest = tf.train.latest_checkpoint(checkpoint_dir)
        model.load_weights(latest)
    except:
        x = 0
    model.fit(batch.drop(columns=['winner']),
         batch['winner'],batch_size=128,epochs=5,callbacks=[cp_callback])
    del(model)

In [27]:
i = 1
for batch_iterator in range(len(batched)):
    batch = batched[0]
    batch['moves'] = batch['moves'].apply(lambda x: splitStandardize(x))
    clear_output()
    print(f'Batch {i} Moves Created')
    createTurns(batch)
    print(f'Batch {i} Turns Created\n')
    batch = batch.drop(columns=['temp','moves','AN'])
    fitBatch(batch)
    del(batch)
    del(batched[0])
    i += 1

Batch 15 Moves Created
Batch 15 Turns Created



MemoryError: Unable to allocate 787. MiB for an array with shape (330, 312742) and data type int64

In [ ]:
del(batch)
del(batch_iterator)
test['moves'] = test['moves'].apply(lambda x: splitStandardize(x))
createTurns(test)
test = test.drop(columns=['temp','moves','AN'])

In [ ]:
model.evaluate(test.drop(columns=['winner','AN']),
         test['winner'],batch_size=128)